<a href="https://colab.research.google.com/github/Shujaat123/Weather_Classification/blob/master/Weather_Classification_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **WC-LSE: Weather Classification From Natural Color Images Using Deep Latent Space Encoding**


This code provide python implementation of WC-LSE algorithm.

# Loading Useful packages

In [ ]:
## Load useful packages
!pip install wget
from random import sample

import keras
# import os.path
from os import path
import h5py
import keras.backend as K
import numpy as np
import tensorflow as tf
import wget
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras.models import Model
from keras.models import load_model
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix

# Pre-processing wheather-classification dataset

**Loading and processing dataset**

In [ ]:
classes = ('cloudy','rain','shine','sunrise')
data_path = 'https://raw.githubusercontent.com/Shujaat123/Weather_Classification/master/dataset/'

flist = []
for fname in classes:
  filename = 'WeatherClassificationDB_'+fname+'.mat'
  if(path.exists(filename)):
    !rm $filename
    print('existing file:', filename, ' has been deleted')
  print('downloading latest version of file:', filename)
  file_path = data_path + filename
  wget.download(file_path, filename)
  print('DONE')
  flist.append(filename)



downloading latest version of file: WeatherClassificationDB_cloudy.mat
DONE
downloading latest version of file: WeatherClassificationDB_rain.mat
DONE
downloading latest version of file: WeatherClassificationDB_shine.mat
DONE
downloading latest version of file: WeatherClassificationDB_sunrise.mat
DONE


In [ ]:
cloudy_imgs = h5py.File(flist[0], 'r')['images']['input']
cloudy_labels = h5py.File(flist[0], 'r')['images']['label']

rain_imgs = h5py.File(flist[1], 'r')['images']['input']
rain_labels = h5py.File(flist[1], 'r')['images']['label']

shine_imgs = h5py.File(flist[2], 'r')['images']['input']
shine_labels = h5py.File(flist[2], 'r')['images']['label']

sunrise_imgs = h5py.File(flist[3], 'r')['images']['input']
sunrise_labels = h5py.File(flist[3], 'r')['images']['label']


In [ ]:
print(cloudy_imgs.shape) # (NHWC)
print(cloudy_labels.shape)

print(rain_imgs.shape) # (NHWC)
print(rain_labels.shape)

print(shine_imgs.shape) # (NHWC)
print(shine_labels.shape)

print(sunrise_imgs.shape) # (NHWC)
print(sunrise_labels.shape)


InputImages = np.concatenate((cloudy_imgs,rain_imgs,shine_imgs,sunrise_imgs), axis = 0)
InputImages = InputImages/InputImages.max()
ClassLabels = np.concatenate((cloudy_labels,rain_labels,shine_labels,sunrise_labels), axis = 0)

InputImages.shape
ClassLabels.shape
# ClassLabels = to_categorical(np.int8(np.squeeze(ClassLabels))-1)
ClassLabels = to_categorical(np.squeeze(ClassLabels)-1)
ClassLabels.shape

(207, 256, 256, 3)
(207, 1, 1, 1)
(215, 256, 256, 3)
(215, 1, 1, 1)
(253, 256, 256, 3)
(253, 1, 1, 1)
(357, 256, 256, 3)
(357, 1, 1, 1)


(1032, 4)

In [ ]:
cloudy_list = list(np.asarray(np.where(ClassLabels.argmax(axis=1)==0)).flatten())
rain_list = list(np.asarray(np.where(ClassLabels.argmax(axis=1)==1)).flatten())
shine_list = list(np.asarray(np.where(ClassLabels.argmax(axis=1)==2)).flatten())
sunrise_list = list(np.asarray(np.where(ClassLabels.argmax(axis=1)==3)).flatten())
total_list = cloudy_list + rain_list + shine_list + sunrise_list

print('Number of \'cloudy\' samples:',len(cloudy_list))
print('Number of \'rain\' samples:',len(rain_list))
print('Number of \'shine\' samples:',len(shine_list))
print('Number of \'sunrise\' samples:',len(sunrise_list))
print('Total number of samples:',len(total_list))

Number of 'cloudy' samples: 207
Number of 'rain' samples: 215
Number of 'shine' samples: 253
Number of 'sunrise' samples: 357
Total number of samples: 1032


**Generate Training, Validation and Test datasets**

In [ ]:
## train select 150 cloudy, 150 rain, 150 shine and 150 sunrise samples
cloudy_train = sample(cloudy_list, 150)
rain_train = sample(rain_list, 150)
shine_train = sample(shine_list, 150)
sunrise_train = sample(sunrise_list, 150)
train_list = cloudy_train + rain_train + shine_train + sunrise_train

Input_train = InputImages[train_list]
Label_train = ClassLabels[train_list]

# valid select 20 cloudy, 20 rain, 20 shine and 20 sunrise samples
cloudy_val = sample(set(cloudy_list) - set(cloudy_train), 20)
rain_val = sample(set(rain_list) - set(rain_train), 20)
shine_val = sample(set(shine_list) - set(shine_train), 20)
sunrise_val = sample(set(sunrise_list) - set(sunrise_train), 20)
val_list = cloudy_val + rain_val + shine_val + sunrise_val

Input_val = InputImages[val_list]
Label_val = ClassLabels[val_list]

## test
test_list = list(set(total_list) - set(train_list) - set(val_list))

# test_list
Input_test = InputImages[test_list]
Label_test = ClassLabels[test_list]

In [ ]:
print(70*'-','\nTraining Dataset distribution\n',70*'-')
print('Number of \'cloudy\' samples:',len(cloudy_train))
print('Number of \'rain\' samples:',len(rain_train))
print('Number of \'shine\' samples:',len(shine_train))
print('Number of \'sunrise\' samples:',len(sunrise_train))
print('Total number of samples:',len(train_list))

print(70*'-','\nValidation Dataset distribution\n',70*'-')
print('Number of \'cloudy\' samples:',len(cloudy_val))
print('Number of \'rain\' samples:',len(rain_val))
print('Number of \'shine\' samples:',len(shine_val))
print('Number of \'sunrise\' samples:',len(sunrise_val))
print('Total number of samples:',len(val_list))

print(70*'-','\nTest Dataset distribution\n',70*'-')
print('Number of \'cloudy\' samples:',len(cloudy_list)-len(cloudy_train)-len(cloudy_val))
print('Number of \'rain\' samples:',len(rain_list)-len(rain_train)-len(rain_val))
print('Number of \'shine\' samples:',len(shine_list)-len(shine_train)-len(shine_val))
print('Number of \'sunrise\' samples:',len(sunrise_list)-len(sunrise_train)-len(sunrise_val))
print('Total number of samples:',len(test_list))

---------------------------------------------------------------------- 
Training Dataset distribution
 ----------------------------------------------------------------------
Number of 'cloudy' samples: 150
Number of 'rain' samples: 150
Number of 'shine' samples: 150
Number of 'sunrise' samples: 150
Total number of samples: 600
---------------------------------------------------------------------- 
Validation Dataset distribution
 ----------------------------------------------------------------------
Number of 'cloudy' samples: 20
Number of 'rain' samples: 20
Number of 'shine' samples: 20
Number of 'sunrise' samples: 20
Total number of samples: 80
---------------------------------------------------------------------- 
Test Dataset distribution
 ----------------------------------------------------------------------
Number of 'cloudy' samples: 37
Number of 'rain' samples: 45
Number of 'shine' samples: 83
Number of 'sunrise' samples: 187
Total number of samples: 352


**Define loss function**

In [ ]:
def loss_DSSIM(y_true, y_pred):
    ssim =tf.image.ssim(y_true, y_pred, 1.0)
    # tv = tf.image.total_variation(y_true - y_pred)
    mae = K.mean(tf.keras.losses.mean_absolute_error(y_true, y_pred))
    return K.mean((1.0 - ssim + mae))

def DSSIM(y_true, y_pred):
    ssim =tf.image.ssim(y_true, y_pred, 1.0)
    return K.mean(ssim)

def DPSNR(y_true, y_pred):
    psnr =tf.image.psnr(y_true, y_pred,1)
    return K.mean(psnr)

def dsc(y_true, y_pred):
    smooth_ = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth_) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth_)
    return score

# def dice_loss(y_true, y_pred):
    # loss = 1 - dsc(y_true, y_pred)
    # return loss

def dice_loss(y_true, y_pred):
  numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
  denominator = tf.reduce_sum(y_true + y_pred, axis=-1)

  return 1 - (numerator + 1) / (denominator + 1)

**Define performance measures**

In [ ]:
def pmeasure(y_true, y_pred):
  cm = confusion_matrix(y_true.ravel(), y_pred.ravel())
  tp_0 = cm[0][0]
  tp_1 = cm[1][1]
  tp_2 = cm[2][2]
  tp_3 = cm[3][3]

  tn_0 = cm[1][1] + cm[1][2] + cm[1][3] + cm[2][1] + cm[2][2] + cm[2][3] + cm[3][1] + cm[3][2] + cm[3][3]
  tn_1 = cm[0][0] + cm[0][2] + cm[0][3] + cm[2][0] + cm[2][2] + cm[2][3] + cm[3][0] + cm[3][2] + cm[3][3]
  tn_2 = cm[0][0] + cm[0][1] + cm[0][3] + cm[1][0] + cm[1][1] + cm[1][3] + cm[3][0] + cm[3][1] + cm[3][3]
  tn_3 = cm[0][0] + cm[0][1] + cm[0][2] + cm[1][0] + cm[1][1] + cm[1][2] + cm[2][0] + cm[2][1] + cm[2][2]

  fp_0 = cm[1][0] + cm[2][0] + cm[3][0]
  fp_1 = cm[0][1] + cm[2][1] + cm[3][1]
  fp_2 = cm[0][2] + cm[1][2] + cm[3][2]
  fp_3 = cm[0][3] + cm[1][3] + cm[2][3]

  fn_0 = cm[0][1] + cm[0][2] + cm[0][3]
  fn_1 = cm[1][0] + cm[1][2] + cm[1][3]
  fn_2 = cm[2][0] + cm[2][1] + cm[2][3]
  fn_3 = cm[3][0] + cm[3][1] + cm[3][2]

  sensitivity_0 = tp_0 / (tp_0 + fn_0)
  sensitivity_1 = tp_1 / (tp_1 + fn_1)
  sensitivity_2 = tp_2 / (tp_2 + fn_2)
  sensitivity_3 = tp_3 / (tp_3 + fn_3)

  specificity_0 = tn_0 / (tn_0 + fp_0)
  specificity_1 = tn_1 / (tn_1 + fp_1)
  specificity_2 = tn_2 / (tn_2 + fp_2)
  specificity_3 = tn_3 / (tn_3 + fp_3)

  f1_score_0 = 2 * tp_0 / (2 * (tp_0 + fp_0 + fn_0))
  f1_score_1 = 2 * tp_1 / (2 * (tp_1 + fp_1 + fn_1))
  f1_score_2 = 2 * tp_2 / (2 * (tp_2 + fp_2 + fn_2))
  f1_score_3 = 2 * tp_3 / (2 * (tp_3 + fp_3 + fn_3))


  return ({'Cloudy Sensitivity': sensitivity_0, 'Rain Sensitivity': sensitivity_1,
           'Shine Sensitivity': sensitivity_2,'Sunrise Sensitivity': sensitivity_3,
           'Cloudy Specificity': specificity_0, 'Rain Specificity': specificity_1,
           'Shine Specificity': specificity_2,'Sunrise Specificity': specificity_3,
           'Cloudy F1-Score': f1_score_0, 'Rain F1-Score': f1_score_1,
           'Shine F1-Score': f1_score_1,'Sunrise F1-Score': f1_score_3})
  
def check_preds(ypred, ytrue):
    smooth = 1
    pred = np.ndarray.flatten(np.clip(ypred, 0, 1))
    gt = np.ndarray.flatten(np.clip(ytrue, 0, 1))
    intersection = np.sum(pred * gt) 
    union = np.sum(pred) + np.sum(gt)   
    return np.round((2 * intersection + smooth)/(union + smooth), decimals=5)

def auc(y_true, y_pred):
    smooth = 1
    y_pred_pos = np.round(np.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = np.round(np.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    tp = np.sum(y_pos * y_pred_pos)
    tn = np.sum(y_neg * y_pred_neg)
    fp = np.sum(y_neg * y_pred_pos)
    fn = np.sum(y_pos * y_pred_neg)
    tpr = (tp + smooth) / (tp + fn + smooth)  # recall
    tnr = (tn + smooth) / (tn + fp + smooth)
    prec = (tp + smooth) / (tp + fp + smooth)  # precision
    return [tpr, tnr, prec]
  
def dice_score(y_true, y_pred, thres):
  dice = np.zeros(y_true.shape[0])
  dsc_thres = np.zeros_like(dice)
  recall = np.zeros_like(dice)
  precision = np.zeros_like(dice)

  for i in range(y_true.shape[0]):
    ds = dsc(y_true[i], y_pred[i])
    dice[i] = K.eval(ds)
    dsc_thres[i] = check_preds(y_pred[i] > thres, y_true[i])
    recall[i], _, precision[i] = auc(y_true[i], y_pred[i] > thres)
  
  dice = np.mean(dice)
  dsc_thres = np.mean(dsc_thres)
  recall = np.mean(recall)
  precision = np.mean(precision)

  return {'Dice score': dice, 'Dice score with threshold': dsc_thres, 'Recall': recall, 'Precision': precision}

def Show_Statistics(msg, Stat):
    print(msg.upper())
    print(70 * '-')
    print('Accuracy:', Stat[0])

    print('Cloudy Sensitivity:', Stat[1])
    print('Rain Sensitivity:', Stat[2])
    print('Shine Sensitivity:', Stat[3])
    print('Sunrise Sensitivity:', Stat[4])

    print('Cloudy Specificity:', Stat[5])
    print('Rain Specificity:', Stat[6])
    print('Shine Specificity:', Stat[7])
    print('Sunrise Specificity:', Stat[8])

    print('Cloudy F1-Score:', Stat[9])
    print('Rain F1-Score:', Stat[10])
    print('Shine F1-Score:', Stat[11])
    print('Sunrise F1-Score:', Stat[12])

    print('Balance Accuracy:', Stat[13])
    print(70 * '-')



# Designing an Auto-Encoder-based classification model

In [ ]:
# def BUS_Final_Model(num_filters=64,input_shape=(256, 256, 3)):
#     # Encoder Network
#     encoder_input = Input(shape=input_shape, name='encoder_input')
#     enc_l1 = Conv2D(num_filters, 15, activation='relu', name='encoder_layer1', padding='same')(encoder_input)
#     enc_l1 = BatchNormalization()(enc_l1)
#     enc_l1 = Conv2D(num_filters, 3, activation='relu', name='encoder_layer2', padding='same')(enc_l1)
#     enc_l1 = BatchNormalization()(enc_l1)
#     # enc_l1 = Dropout(0.4)(enc_l1)
#     enc_l1 = MaxPooling2D(pool_size=(2, 2))(enc_l1)

#     enc_l2 = Conv2D(2 * num_filters, 3, activation='relu', name='encoder_layer3', padding='same')(enc_l1)
#     enc_l2 = BatchNormalization()(enc_l2)
#     enc_l2 = Conv2D(2 * num_filters, 3, activation='relu', name='encoder_layer4', padding='same')(enc_l2)
#     enc_l2 = BatchNormalization()(enc_l2)
#     # enc_l2 = Dropout(0.4)(enc_l2)
#     enc_l2 = MaxPooling2D(pool_size=(2, 2))(enc_l2)

#     enc_l3 = Conv2D(4 * num_filters, 3, activation='relu', name='encoder_layer5', padding='same')(enc_l2)
#     enc_l3 = BatchNormalization()(enc_l3)
#     enc_l3 = Conv2D(4 * num_filters, 3, activation='relu', name='encoder_layer6', padding='same')(enc_l3)
#     enc_l3 = BatchNormalization()(enc_l3)
#     # enc_l3 = Dropout(0.4)(enc_l3)
#     enc_l3 = MaxPooling2D(pool_size=(2, 2))(enc_l3)

#     enc_l4 = Conv2D(8 * num_filters, 3, activation='relu', name='encoder_layer7', padding='same')(enc_l3)
#     enc_l4 = BatchNormalization()(enc_l4)
#     enc_l4 = Conv2D(8 * num_filters, 3, activation='relu', name='encoder_layer8', padding='same')(enc_l4)
#     enc_l4 = BatchNormalization()(enc_l4)
#     # enc_l4 = Dropout(0.4)(enc_l4)
#     enc_l4 = MaxPooling2D(pool_size=(2, 2))(enc_l4)

#     enc_l5 = Conv2D(16 * num_filters, 3, activation='relu', name='encoder_layer9', padding='same')(enc_l4)
#     enc_l5 = BatchNormalization()(enc_l5)
#     enc_l5 = Conv2D(16 * num_filters, 3, activation='relu', name='encoder_layer10', padding='same')(enc_l5)
#     enc_l5 = BatchNormalization()(enc_l5)
#     enc_l5 = Dropout(0.4)(enc_l5)
#     enc_l5 = MaxPooling2D(pool_size=(2, 2))(enc_l5)

#     encoder_output = Conv2D(32 * num_filters, 3, activation='relu', name='encoder_output', padding='same')(enc_l5)

#     # Classifier Network
#     flat = Flatten()(encoder_output)
#     class_l1 = Dense(1024, activation='relu', name='class_layer1')(flat)
#     class_l1 = BatchNormalization()(class_l1)
#     drop_c1 = Dropout(0.4)(class_l1)
#     class_l2 = Dense(512, activation='relu', name='class_layer2')(drop_c1)
#     class_l2 = BatchNormalization()(class_l2)
#     drop_c2 = Dropout(0.4)(class_l2)
#     class_l3 = Dense(256, activation='relu', name='class_layer3')(drop_c2)
#     class_l3 = BatchNormalization()(class_l3)
#     drop_c3 = Dropout(0.4)(class_l3)
#     class_output = Dense(4, activation='softmax', name='class_output')(drop_c3)

#     # Decoder Network
#     # dec_l1 = UpSampling2D(size=(2, 2))(concatenate([enc_l5, encoder_output], axis=3))
#     dec_l1 = UpSampling2D(size=(2, 2))(encoder_output)
#     dec_l1 = Conv2D(16 * num_filters, 3, activation='relu', name='decoder_layer1', padding='same')(dec_l1)
#     dec_l1 = BatchNormalization()(dec_l1)
#     dec_l1 = Conv2D(16 * num_filters, 3, activation='relu', name='decoder_layer2', padding='same')(dec_l1)
#     dec_l1 = BatchNormalization()(dec_l1)
#     # dec_l1 = Dropout(0.4)(dec_l1)

#     # dec_l2 = UpSampling2D(size=(2, 2))(concatenate([enc_l4, dec_l1], axis=3))
#     dec_l2 = UpSampling2D(size=(2, 2))(dec_l1)
#     dec_l2 = Conv2D(8 * num_filters, 3, activation='relu', name='decoder_layer3', padding='same')(dec_l2)
#     # dec_l2 = BatchNormalization()(dec_l2)
#     dec_l2 = Conv2D(8 * num_filters, 3, activation='relu', name='decoder_layer4', padding='same')(dec_l2)
#     dec_l2 = BatchNormalization()(dec_l2)
#     # dec_l2 = Dropout(0.4)(dec_l2)

#     # dec_l3 = UpSampling2D(size=(2, 2))(concatenate([enc_l3, dec_l2], axis=3))
#     dec_l3 = UpSampling2D(size=(2, 2))(dec_l2)
#     dec_l3 = Conv2D(2 * num_filters, 3, activation='relu', name='decoder_layer5', padding='same')(dec_l3)
#     # dec_l3 = BatchNormalization()(dec_l3)
#     dec_l3 = Conv2D(2 * num_filters, 3, activation='relu', name='decoder_layer6', padding='same')(dec_l3)
#     dec_l3 = BatchNormalization()(dec_l3)
#     # dec_l3 = Dropout(0.4)(dec_l3)

#     # dec_l4 = UpSampling2D(size=(2, 2))(concatenate([enc_l2, dec_l3], axis=3))
#     dec_l4 = UpSampling2D(size=(2, 2))(dec_l3)
#     dec_l4 = Conv2D(num_filters, 3, activation='relu', name='decoder_layer7', padding='same')(dec_l4)
#     # dec_l4 = BatchNormalization()(dec_l4)
#     dec_l4 = Conv2D(num_filters, 3, activation='relu', name='decoder_layer8', padding='same')(dec_l4)
#     dec_l4 = BatchNormalization()(dec_l4)
#     # dec_l4 = Dropout(0.4)(dec_l4)

#     # dec_l5 = UpSampling2D(size=(2, 2))(concatenate([enc_l1, dec_l4], axis=3))
#     dec_l5 = UpSampling2D(size=(2, 2))(dec_l4)
#     dec_l5 = Conv2D(num_filters, 3, activation='relu', name='decoder_layer10', padding='same')(dec_l5)
#     # dec_l5 = BatchNormalization()(dec_l5)
#     dec_l5 = Conv2D(num_filters, 3, activation='relu', name='decoder_layer9', padding='same')(dec_l5)
#     dec_l5 = BatchNormalization()(dec_l5)
#     dec_l5 = Dropout(0.4)(dec_l5)

#     decoder_output = Conv2D(input_shape[2], 1, activation='relu', name='decoder_output', padding='same')(dec_l5)

#     model = Model(inputs=[encoder_input], outputs=[class_output, decoder_output])  # class_output, decoder_output,

#     # Compiling model
#     model.compile(optimizer=tf.keras.optimizers.Adam(),#learning_rate=0.0001, beta_1=0.9, beta_2=0.999,
#                                                       #epsilon=1e-07, amsgrad=False),
#                   loss= {'class_output': 'categorical_crossentropy', 'decoder_output': 'mae'}, #loss_DSSIM},
#                   loss_weights={'class_output': 0.01,  'decoder_output': 0.99},
#                   metrics={'class_output': 'categorical_accuracy',  'decoder_output': 'mae'})#[DSSIM, DPSNR, 'mae']})
#     return model

In [ ]:
def BUS_Final_Model(input_shape=(256, 256, 3)):
  base_model = keras.applications.Xception(
      weights='imagenet',  # Load weights pre-trained on ImageNet.
      # weights=None,  # Load weights pre-trained on ImageNet.
      input_shape=input_shape,
      include_top=False)  # Do not include the ImageNet classifier at the top.
  base_model.trainable = False
  # Encoder Network
  encoder_input = Input(shape=input_shape, name='encoder_input')
  # We make sure that the base_model is running in inference mode here,
  # by passing `training=False`. This is important for fine-tuning, as you will
  # learn in a few paragraphs.
  x = base_model(encoder_input, training=False)
  # Convert features of shape `base_model.output_shape[1:]` to vectors
  x = keras.layers.GlobalAveragePooling2D()(x)
  # A Dense classifier with a single unit (binary classification)
  class_output = keras.layers.Dense(4, name='class_output')(x)
  model = keras.Model(encoder_input, class_output)

  model.compile(optimizer=keras.optimizers.Adam(),#lr=0.0001),
                loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
                # metrics=[keras.metrics.BinaryAccuracy()])
  model.summary()
  return model


In [ ]:
# model = LSE_model(input_size=Bmode_train.shape[1:],filters=8,kernel_size=3,upconv=False,droprate=0.4, batchnorm=2)
model = BUS_Final_Model()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 256, 256, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 8, 8, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
class_output (Dense)         (None, 4)                 8196      
Total params: 20,869,676
Trainable params: 8,196
Non-trainable params: 20,861,480
_________________________________________________________________


# Model Training

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
checkpoint = ModelCheckpoint('models\\model-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

# es = EarlyStopping(monitor='val_class_output_categorical_accuracy', mode='max', verbose=0, patience=10)

# checkpoint = ModelCheckpoint('models\\model-best.h5', verbose=1,
#                              monitor='val_class_output_categorical_accuracy',
#                              save_best_only=True, mode='auto')
  
# history = model.fit({'encoder_input': Input_train}, {'class_output': Label_train,'decoder_output': Input_train},             
#                     validation_data=({'encoder_input': Input_val}, {'class_output': Label_val, 'decoder_output': Input_val}),                                      
#                     batch_size=10, epochs=100, shuffle=True, verbose=2, callbacks=[checkpoint, es])

history = model.fit({'encoder_input': Input_train}, {'class_output': Label_train},
                    validation_data=({'encoder_input': Input_val}, {'class_output': Label_val}),
                    batch_size=1, epochs=100, shuffle=True, verbose=2, callbacks=[checkpoint, es])


Epoch 1/100

Epoch 00001: val_loss improved from inf to 0.40021, saving model to models\model-best.h5
600/600 - 7s - loss: 0.6241 - accuracy: 0.7900 - val_loss: 0.4002 - val_accuracy: 0.9000
Epoch 2/100

Epoch 00002: val_loss improved from 0.40021 to 0.30337, saving model to models\model-best.h5
600/600 - 7s - loss: 0.2696 - accuracy: 0.9283 - val_loss: 0.3034 - val_accuracy: 0.9250
Epoch 3/100

Epoch 00003: val_loss improved from 0.30337 to 0.28193, saving model to models\model-best.h5
600/600 - 7s - loss: 0.1897 - accuracy: 0.9367 - val_loss: 0.2819 - val_accuracy: 0.9250
Epoch 4/100

Epoch 00004: val_loss improved from 0.28193 to 0.24655, saving model to models\model-best.h5
600/600 - 7s - loss: 0.1338 - accuracy: 0.9617 - val_loss: 0.2466 - val_accuracy: 0.9250
Epoch 5/100

Epoch 00005: val_loss improved from 0.24655 to 0.21946, saving model to models\model-best.h5
600/600 - 7s - loss: 0.1015 - accuracy: 0.9817 - val_loss: 0.2195 - val_accuracy: 0.9125
Epoch 6/100

Epoch 00006: val

# Performance Evaluation

In [ ]:
del model
model = load_model('models\model-best.h5')              
# [Label_train_pred, decoder_output_train_pred] = model.predict(Input_train, batch_size=1, verbose=0)
# [Label_val_pred, decoder_output_val_pred] = model.predict(Input_val, batch_size=1, verbose=0)
# [Label_test_pred, decoder_output_test_pred] = model.predict(Input_test, batch_size=1, verbose=0)
Label_train_pred = model.predict(Input_train, batch_size=1, verbose=0)
Label_val_pred = model.predict(Input_val, batch_size=1, verbose=0)
Label_test_pred = model.predict(Input_test, batch_size=1, verbose=0)

In [ ]:
Training_Stats = []
# Training Measures
tr_acc = accuracy_score(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))
tr_sen_a = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Cloudy Sensitivity']
tr_sen_b = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Rain Sensitivity']
tr_sen_c = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Shine Sensitivity']
tr_sen_d = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Sunrise Sensitivity']

tr_spe_a = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Cloudy Specificity']
tr_spe_b = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Rain Specificity']
tr_spe_c = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Shine Specificity']
tr_spe_d = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Sunrise Specificity']

tr_f1_a = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Cloudy F1-Score']
tr_f1_b = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Rain F1-Score']
tr_f1_c = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Shine F1-Score']
tr_f1_d = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Sunrise F1-Score']
tr_bacc = balanced_accuracy_score(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))

Training_Stats.append([tr_acc,
                       tr_sen_a, tr_sen_b, tr_sen_c, tr_sen_d,
                       tr_spe_a, tr_spe_b, tr_spe_c, tr_spe_d,
                       tr_f1_a, tr_f1_b, tr_f1_c, tr_f1_d,
                       tr_bacc])


In [ ]:
Validation_Stats = []
# Validation Measures

v_acc = accuracy_score(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))
v_sen_a = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Cloudy Sensitivity']
v_sen_b = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Rain Sensitivity']
v_sen_c = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Shine Sensitivity']
v_sen_d = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Sunrise Sensitivity']

v_spe_a = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Cloudy Specificity']
v_spe_b = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Rain Specificity']
v_spe_c = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Shine Specificity']
v_spe_d = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Sunrise Specificity']

v_f1_a = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Cloudy F1-Score']
v_f1_b = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Rain F1-Score']
v_f1_c = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Shine F1-Score']
v_f1_d = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Sunrise F1-Score']
v_bacc = balanced_accuracy_score(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))

Validation_Stats.append([v_acc,
                  v_sen_a, v_sen_b, v_sen_c, v_sen_d,
                  v_spe_a, v_spe_b, v_spe_c, v_spe_d,
                  v_f1_a, v_f1_b, v_f1_c, v_f1_d,
                  v_bacc])

In [ ]:
Testing_Stats = []
# Test Measures
t_acc = accuracy_score(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))
t_sen_a = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Cloudy Sensitivity']
t_sen_b = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Rain Sensitivity']
t_sen_c = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Shine Sensitivity']
t_sen_d = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Sunrise Sensitivity']

t_spe_a = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Cloudy Specificity']
t_spe_b = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Rain Specificity']
t_spe_c = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Shine Specificity']
t_spe_d = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Sunrise Specificity']

t_f1_a = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Cloudy F1-Score']
t_f1_b = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Rain F1-Score']
t_f1_c = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Shine F1-Score']
t_f1_d = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Sunrise F1-Score']
t_bacc = balanced_accuracy_score(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))

Testing_Stats.append([t_acc,
                      t_sen_a, t_sen_b, t_sen_c, t_sen_d,
                      t_spe_a, t_spe_b, t_spe_c, t_spe_d,
                      t_f1_a, t_f1_b, t_f1_c, t_f1_d,
                      t_bacc]) 

In [ ]:
Train_Statistics = np.asarray(Training_Stats)
Val_Statistics = np.asarray(Validation_Stats)
Test_Statistics = np.asarray(Testing_Stats)

# Show Classification/Reconstruction Statistics
Show_Statistics('Training Results', Train_Statistics[0])
Show_Statistics('Validation Results', Val_Statistics[0])
Show_Statistics('Test Results', Test_Statistics[0])

TRAINING RESULTS
----------------------------------------------------------------------
Accuracy: 0.9983333333333333
Cloudy Sensitivity: 1.0
Rain Sensitivity: 1.0
Shine Sensitivity: 0.9933333333333333
Sunrise Sensitivity: 1.0
Cloudy Specificity: 0.9977777777777778
Rain Specificity: 1.0
Shine Specificity: 1.0
Sunrise Specificity: 1.0
Cloudy F1-Score: 0.9933774834437086
Rain F1-Score: 1.0
Shine F1-Score: 1.0
Sunrise F1-Score: 1.0
Balance Accuracy: 0.9983333333333333
----------------------------------------------------------------------
VALIDATION RESULTS
----------------------------------------------------------------------
Accuracy: 0.9
Cloudy Sensitivity: 0.8
Rain Sensitivity: 0.95
Shine Sensitivity: 0.9
Sunrise Sensitivity: 0.95
Cloudy Specificity: 0.9666666666666667
Rain Specificity: 1.0
Shine Specificity: 0.9333333333333333
Sunrise Specificity: 0.9666666666666667
Cloudy F1-Score: 0.7272727272727273
Rain F1-Score: 0.95
Shine F1-Score: 0.95
Sunrise F1-Score: 0.8636363636363636
Balance